<a href="https://colab.research.google.com/github/zymoide/GPT2_Friends_Text_Generation/blob/main/Text_Generation_Project_LSTM_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [ ]:
read_file("/content/Friends_S_1.txt")

In [3]:
import spacy


In [4]:
nlp = spacy.load('en',disable=['parser', 'tagger','ner'])
nlp.max_length = 5198623

# New Section

In [5]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [6]:
d = read_file('/content/Friends_S_1.txt')
tokens = separate_punc(d)

In [ ]:
tokens

In [8]:
len(tokens)

98435

In [9]:
# organize into sequences of tokens
train_len = 25+1 # 50 training words, then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [10]:
' '.join(text_sequences[0])

'\ufeffthe one where monica gets a new roommate the pilot the uncut version written by marta kauffman david crane transcribed by guineapig additional transcribing by eric'

In [ ]:
' '.join(text_sequences[1])

In [ ]:
' '.join(text_sequences[2])

In [13]:
len(text_sequences)

98409

In [14]:
from keras.preprocessing.text import Tokenizer

In [15]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
sequences[0]

In [ ]:
tokenizer.index_word

In [ ]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

In [ ]:
tokenizer.word_counts

In [20]:
vocabulary_size = len(tokenizer.word_counts)

In [21]:
vocabulary_size

6121

In [22]:
import numpy as np

In [23]:
sequences = np.array(sequences)

In [24]:
sequences

array([[6121,   71,  137, ..., 6116,   90, 2341],
       [  71,  137,   10, ...,   90, 2341, 2342],
       [ 137,   10,  220, ..., 2341, 2342,  882],
       ...,
       [  76,   77,  165, ...,    3,  881,   30],
       [  77,  165,    2, ...,  881,   30,  294],
       [ 165,    2,  227, ...,   30,  294,  132]])

In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding


In [26]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(550, return_sequences=True))
    model.add(LSTM(550))
    model.add(Dense(550, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [27]:
from keras.utils import to_categorical

In [ ]:
sequences

In [ ]:
# First 49 words
sequences[:,:-1]

In [30]:
# last Word
sequences[:,-1]

array([2341, 2342,  882, ...,   30,  294,  132])

In [31]:
X = sequences[:,:-1]

In [32]:
y = sequences[:,-1]

In [33]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [ ]:
y

In [35]:
seq_len = X.shape[1]

In [36]:
seq_len

25

In [37]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            153050    
_________________________________________________________________
lstm (LSTM)                  (None, 25, 550)           1267200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 550)               2422200   
_________________________________________________________________
dense (Dense)                (None, 550)               303050    
_________________________________________________________________
dense_1 (Dense)              (None, 6122)              3373222   
Total params: 7,518,722
Trainable params: 7,518,722
Non-trainable params: 0
_________________________________________________________________


In [38]:
from pickle import dump,load

In [39]:
model.fit(X, y, batch_size=128, epochs=100,verbose=1)


Epoch 1/100
769/769 [==============================] - 40s 28ms/step - loss: 6.4682 - accuracy: 0.0311
Epoch 2/100
769/769 [==============================] - 21s 28ms/step - loss: 5.9088 - accuracy: 0.0483
Epoch 3/100
769/769 [==============================] - 21s 28ms/step - loss: 5.6650 - accuracy: 0.0671
Epoch 4/100
769/769 [==============================] - 21s 27ms/step - loss: 5.4460 - accuracy: 0.0875
Epoch 5/100
769/769 [==============================] - 21s 27ms/step - loss: 5.2687 - accuracy: 0.1023
Epoch 6/100
769/769 [==============================] - 21s 27ms/step - loss: 5.1109 - accuracy: 0.1132
Epoch 7/100
769/769 [==============================] - 21s 27ms/step - loss: 4.9682 - accuracy: 0.1232
Epoch 8/100
769/769 [==============================] - 21s 27ms/step - loss: 4.9206 - accuracy: 0.1243
Epoch 9/100
769/769 [==============================] - 21s 27ms/step - loss: 4.7751 - accuracy: 0.1345
Epoch 10/100
769/769 [==============================] - 21s 27ms/step - l

In [40]:
# save the model to file
model.save('epochBIG.h5')
# save the tokenizer
dump(tokenizer, open('epochBIG', 'wb'))

In [41]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [42]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [43]:
text_sequences[0]

['\ufeffthe',
 'one',
 'where',
 'monica',
 'gets',
 'a',
 'new',
 'roommate',
 'the',
 'pilot',
 'the',
 'uncut',
 'version',
 'written',
 'by',
 'marta',
 'kauffman',
 'david',
 'crane',
 'transcribed',
 'by',
 'guineapig',
 'additional',
 'transcribing',
 'by',
 'eric']

In [75]:
import random
#random.seed(101)
random_pick = random.randint(0,len(text_sequences))


In [76]:
#random_seed_text = text_sequences[random_pick]
random_seed_text = text_sequences[random_pick] #continuing my try


In [77]:
random_seed_text

['they',
 'both',
 'leave',
 'just',
 'as',
 'rachel',
 'enters',
 'the',
 'room',
 'holding',
 'a',
 'cup',
 'rachel',
 'hi',
 'i',
 'thought',
 'you',
 'might',
 'like',
 'some',
 'ice',
 'chips',
 'carol',
 'thanks',
 'rachel',
 'and']

In [78]:
seed_text = ' '.join(random_seed_text)

In [79]:
seed_text

'they both leave just as rachel enters the room holding a cup rachel hi i thought you might like some ice chips carol thanks rachel and'

In [83]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


"if you need anything else i—(notices joey so what 's with you 're gon na be alright uh chandler if nobody was your wenus she has all big kind of insecure and not us they know no one scene monica and rachel 's phoebe is smoking at the phone rachel"

In [50]:
full_text = read_file('/content/Friends_S_1.txt')

In [51]:
for i,word in enumerate(full_text.split()):
    if word == 'inkling':
        print(' '.join(full_text.split()[i-20:i+20]))
        print('\n')

In [ ]:
### Code taken (and modified) from Mr J Portilla. ###